# LangGraph Design Patterns

#TODO: give me a brief two-sentence description of the notebook and then an outline of all of sections with a bried desctription of what we do in each of the sections
  

## Set up the Environment and Components

#TODO provide two sentence descripton of the section

In [ ]:
!pip install langchain-core langgraph ipython-extensions typing-extensions langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.4 MB/s eta 0:00:00


In [ ]:
# Import the required libraries
from typing import Annotated
from typing_extensions import TypedDict
import random
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph import MessagesState

from IPython.display import Image, display

Setting up a connection to a proprietary remote model

In [ ]:
# # Set OpenAI API Key
# import os, getpass

# def _set_env(var: str):
#     if not os.environ.get(var):
#         os.environ[var] = getpass.getpass(f"{var}: ")

# _set_env("OPENAI_API_KEY")

OPENAI_API_KEY: ··········


In [ ]:
# @title Loading the `config.json` file
import json
from langchain_openai import ChatOpenAI
import os

# Load the JSON file and extract values
file_name = 'config.json'
with open(file_name, 'r') as file:
    config = json.load(file)
    os.environ['OPENAI_API_KEY'] = config.get("API_KEY") # Loading the API Key
    os.environ["OPENAI_BASE_URL"] = config.get("OPENAI_API_BASE") # Loading the API Base Url

In [ ]:
gpt4o_chat = ChatOpenAI(model="gpt-4o", temperature=0)
gpt4o_chat.invoke("hello world")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 9, 'total_tokens': 19, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_6ec83003ad', 'id': 'chatcmpl-BEcMQYNSwNKVOq9IMeCwzcbW1N6id', 'finish_reason': 'stop', 'logprobs': None}, id='run-86ac577b-d70e-4406-b35e-045096138ff4-0', usage_metadata={'input_tokens': 9, 'output_tokens': 10, 'total_tokens': 19, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Create some tools for the LLM

In [ ]:
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

In [ ]:
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b

In [ ]:
def determine_outcome(bet: int) -> str:
    """Roll a pair of dice and determine wether the sum of that roll is less than the bet

    Args:
        bet: the number that you believe the dice sum will be less than or equal to (number between 1 and 12)
    """
    roll = random.randint(1, 6) + random.randint(1, 6)

    return 'success' if roll <= bet else 'failure'

Bind the tools to the LLM

In [ ]:
llm_with_tools = gpt4o_chat.bind_tools([multiply, determine_outcome])

In [ ]:
tool_call = llm_with_tools.invoke([HumanMessage(content=f"What is 2 multiplied by 3",)])

In [ ]:
tool_call.pretty_print()

## Set up the State

#TODO provide two sentence descripton of the section

In [ ]:
class MessagesState(MessagesState):
    # Add any keys needed beyond messages, which is pre-built
    pass

A brief look at reducers
Reducers allow us to specify how state updates are performed. If no reducer function is specified, then it is assumed that updates to the key should override it as we saw before.

To append messages, we can use the pre-built add_messages reducer.

In [ ]:
# Initial state
initial_messages = [AIMessage(content="Hello! How can I assist you?", name="Model"),
                    HumanMessage(content="I need help understanding probability theory.", name="User")
                   ]

# New message to add
new_message = AIMessage(content="Sure, I can help with that. What specifically are you interested in?", name="Model")

# Test
add_messages(initial_messages , new_message)

## Create a Chain

#TODO provide two sentence descripton of the section

In [ ]:
def tool_calling_llm(state: MessagesState):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# Build graph
builder = StateGraph(MessagesState)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_edge(START, "tool_calling_llm")
builder.add_edge("tool_calling_llm", END)
graph = builder.compile()

# View
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
messages = graph.invoke({"messages": HumanMessage(content="Hello!")})
for m in messages['messages']:
    m.pretty_print()

In [ ]:
messages = graph.invoke({"messages": HumanMessage(content="Multiply 5 and 7")})
for m in messages['messages']:
    m.pretty_print()

In [ ]:
messages = graph.invoke({"messages": HumanMessage(content="Bet 6 against a dice roll")})
for m in messages['messages']:
    m.pretty_print()

## Create a Router

#TODO provide two sentence descripton of the section. Make sure to talk about the LangGraph built in ToolNode and tools_condition for the conditional edge. Also decribe how this one is different than our previous chain

In [ ]:
# Node
def tool_calling_llm(state: MessagesState):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# Build graph
builder = StateGraph(MessagesState)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode([multiply, add, determine_outcome]))
builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", END)
graph = builder.compile()

# View
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
messages = graph.invoke({"messages": HumanMessage(content="Hello!")})
for m in messages['messages']:
    m.pretty_print()

In [ ]:
messages = graph.invoke({"messages": HumanMessage(content="Multiply 5 and 7")})
for m in messages['messages']:
    m.pretty_print()

In [ ]:
messages = graph.invoke({"messages": HumanMessage(content="Bet 6 against a dice roll")})
for m in messages['messages']:
    m.pretty_print()

## Create an Agent

#TODO provide two sentence descripton of the section. Make sure to talk about the agent node we are creating and how this design pattern is different than the router one. Also walkthrough the execution of the graph

In [ ]:
# System message
sys_msg = SystemMessage(content="You are a helpful assistant tasked with performing arithmetic to determine a bet and then resolving wether that bet passes a dice roll test.")

# Node
def assistant(state: MessagesState):
   return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

In [ ]:
# Graph
builder = StateGraph(MessagesState)

# Define nodes: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode([add, multiply, determine_outcome]))

# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", "assistant")
react_graph = builder.compile()

# Show
display(Image(react_graph.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
messages = [HumanMessage(content="My defender has a +2 defense. The attack has a +1 modifier for range and +2 modifier for moving. "
"I need to determine wether my defender passes the attack role by summing together the defense and modifers and then checking the outcome versus a dice roll.")]
messages = react_graph.invoke({"messages": messages})

In [ ]:
for m in messages['messages']:
    m.pretty_print()